# Introduction:
I feel like removing outliers makes results worse. We are better off just using feature engineering alone, with maybe just setting upper bounds for extreme outlier values.

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os
from pathlib import Path
import xgboost as xgb
import lightgbm as lgbm
import catboost
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
from IPython.display import display
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
import optuna
from sklearn.preprocessing import StandardScaler

In [4]:
from warnings import filterwarnings
filterwarnings("ignore")

# Loading Data

In [5]:
# setting a base path variable for easy access
BASE_PATH = Path("/kaggle/input/playground-series-s3e6")
train = pd.read_csv(BASE_PATH / "train.csv").drop(columns=["id"])

test = pd.read_csv(BASE_PATH / "test.csv")
# we need the test id column to make the submission
test_idx = test.id
test = test.drop(columns=["id"])

# we also load original dataset and will try to gauge if adding it to the competition datasets boosts our score or not
original = pd.read_csv("/kaggle/input/paris-housing-price-prediction/ParisHousing.csv")

### Features Presence Check
Checkingg if the competition dataset and original dataset contain the same features in same order, so we can concatenate them easily. Otherwise we'll have to manually make them consistent

In [6]:
all(original.columns == train.columns)

True

#### Result:
Well both dataset indeed contain the same features in the same order. No need for any manual work!

In [7]:
all_datasets = {"train": train, "test": test,"original": original}

# Setting bounds

In [8]:
for dataset in all_datasets.values():
    dataset["attic"] = dataset.attic.clip(upper=10000, lower=10)
    dataset["floors"] = dataset.floors.clip(upper=100)
    dataset["squareMeters"] = dataset.squareMeters.clip(upper=99999)
    dataset["basement"] = dataset.basement.clip(upper=10000, lower=10)
    dataset["garage"] = dataset.garage.clip(upper=1000, lower=100)
    dataset["made"] = dataset.made.clip(upper=2021)
    dataset["cityCode"] = dataset.cityCode.clip(upper=99999)

# Preprocess

In [9]:
def preprocess(datasets:dict, target:str):
    """Preprocess the data
        datasets: Expects a dict with datasets name as keys and datasets as values
            Valid names for dataets are: [train, test, original]
        
        target: name of the target column
    """
    for dataset_name, dataset in datasets.items():
        if dataset_name == "train":
            y = train[target]
            X = train.drop(columns=[target])
        elif dataset_name == "original":
            y_org = original[target]
            X_org = original.drop(columns=[target])
        elif dataset_name == "test":
            X_test = test.copy()
        
        
    X_combined = pd.concat([X, X_org], axis=0)
    y_combined = pd.concat([y, y_org], axis=0)

    return {"train": {"data": X, "target": y}, "original": {"data": X_org,  "target": y_org},
            "combined": {"data": X_combined, "target": y_combined}, "test": {"data": X_test}}    

# Feature Engineering

In [10]:
#  ------------ We need this for our feature number 8 -----------------
train_copy = all_datasets["train"]
train_copy["block_number"] = train_copy.cityCode // 100
# lets find the average house prices for blocks
avg_house_cost_per_block = train_copy.groupby("block_number")["price"].mean().to_dict()
# ---------------------------------------------------------------------------


for _, dataset in all_datasets.items():
    # FeatIdea #1: guestRooms/numberOfRooms
    dataset["guestRooms_/_numberOfRooms"] = dataset.hasGuestRoom / dataset.numberOfRooms
    
    # FeatIdea #2: Number of non guest rooms: NumOfRooms - GuestRooms
    dataset["non_guest_rooms"] = dataset.numberOfRooms - dataset.hasGuestRoom
    
    # FeatIdea #3: Rooms/Floors: Number of rooms per floor
    dataset["NEW_TEST_FEATURE"] = dataset.numberOfRooms // dataset.floors
    
    # FeatIdea #4: Attic/squareMeters:  Attic size to total house size ratio
    dataset["attic_/_squareMeters"] = dataset.attic / dataset.squareMeters
    
    # FeatIdea #5: Basement/squareMeters: Basement size to total house size ratio
    dataset["basement_/_squareMeters"] = dataset.basement / dataset.squareMeters
    
    # FeatIdea #6. Garage/squareMeters: Garage size to total house size ratio
    dataset["garage_/_squareMeters"] = dataset.garage / dataset.squareMeters
    
    # FeatIdea #7: Grouping zip codes
    dataset["block_number"] = dataset.cityCode // 100
    
    # FeatIdea #8: Add avg price per block as a feature.
    dataset["avg_house_price_in_block"] = dataset.block_number.map(avg_house_cost_per_block)

In [11]:
cols_to_drop = ["cityCode", "block_number"]
for _, dataset in all_datasets.items():
    dataset.drop(columns=cols_to_drop, inplace=True)

# Training models

In [12]:
processed_datasets = preprocess(all_datasets, "price")
X_combined, y_combined = processed_datasets["combined"].values()
X_test = processed_datasets["test"]["data"]


X, y = processed_datasets["train"]["data"], processed_datasets["train"]["target"]
X_org, y_org = processed_datasets["original"]["data"], processed_datasets["original"]["target"]

# Tuning XGBoost

In [13]:
# def objective_xgb(trial, X, y, X_original, y_original):
#     params = {
#         'tree_method': "gpu_hist",
#         'n_estimators': trial.suggest_int('n_estimators', 50, 400),
#         'max_depth': trial.suggest_int('max_depth', 2, 20),
#         'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
#         'min_child_weight': trial.suggest_int('min_child_weight', 1, 15),
#         'gamma': trial.suggest_loguniform('gamma', 0.00001, 0.3),
#         'subsample': trial.suggest_float('subsample', 0.2, 1.0, step=0.05),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.2, 1.0, step=0.05),
#         'early_stopping_rounds': trial.suggest_int("early_stoppig_rounds", 40, 100)
#     }
#     # we're gonna train on the combined dataset but, we'll only calculate the validation score only on comp data

#     N_FOLDS = 5
#     all_scores = np.zeros(N_FOLDS)

#     skf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=1337)

#     for fold_id, (train_idx, val_idx) in enumerate(skf.split(X)):
        
#         X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
#         y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
#         X_tr = pd.concat([X_tr, X_original], axis=0)
#         y_tr = pd.concat([y_tr, y_original], axis=0)
        
#         model = xgb.XGBRegressor(**params)
#         model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)
        
#         y_pred = model.predict(X_val)
                
#         rmse = mean_squared_error(y_val, y_pred, squared=False)        
#         all_scores[fold_id] = rmse
    
#     avg_rmse = np.mean(all_scores)
    
#     print(f"Avg RMSE: {avg_rmse}")
    
#     return avg_rmse

# study_xgb = optuna.create_study(study_name="xgboost_tuning", direction="minimize")
# func = lambda trial: objective_xgb(trial, X, y, X_org, y_org)
# study_xgb.optimize(func, n_trials=50)

[I 2023-02-20 19:11:11,942] A new study created in memory with name: xgboost_tuning
[I 2023-02-20 19:11:15,249] Trial 0 finished with value: 245720.95075908146 and parameters: {'n_estimators': 60, 'max_depth': 5, 'learning_rate': 0.06905483990591715, 'min_child_weight': 13, 'gamma': 0.004579524875897699, 'subsample': 0.45, 'colsample_bytree': 0.6000000000000001, 'early_stoppig_rounds': 95}. Best is trial 0 with value: 245720.95075908146.


Avg RMSE: 245720.95075908146


[I 2023-02-20 19:11:17,726] Trial 1 finished with value: 169219.9462123244 and parameters: {'n_estimators': 373, 'max_depth': 3, 'learning_rate': 0.13572208399682192, 'min_child_weight': 11, 'gamma': 0.029081204538277682, 'subsample': 0.8, 'colsample_bytree': 0.9000000000000001, 'early_stoppig_rounds': 60}. Best is trial 1 with value: 169219.9462123244.


Avg RMSE: 169219.9462123244


[I 2023-02-20 19:13:52,465] Trial 2 finished with value: 375173.1221821708 and parameters: {'n_estimators': 184, 'max_depth': 20, 'learning_rate': 0.1803421933978423, 'min_child_weight': 4, 'gamma': 0.000351612535664332, 'subsample': 0.5, 'colsample_bytree': 0.30000000000000004, 'early_stoppig_rounds': 91}. Best is trial 1 with value: 169219.9462123244.


...

Avg RMSE: 155508.1358940178


[I 2023-02-20 19:28:01,034] Trial 48 finished with value: 182280.33855926702 and parameters: {'n_estimators': 335, 'max_depth': 8, 'learning_rate': 0.10343786650873088, 'min_child_weight': 9, 'gamma': 0.00024293689093412196, 'subsample': 0.7, 'colsample_bytree': 0.5, 'early_stoppig_rounds': 82}. Best is trial 36 with value: 153069.70175736025.


Avg RMSE: 182280.33855926702


[I 2023-02-20 19:28:04,224] Trial 49 finished with value: 180531.56852194102 and parameters: {'n_estimators': 363, 'max_depth': 4, 'learning_rate': 0.11808387289270539, 'min_child_weight': 13, 'gamma': 0.0007089656994741378, 'subsample': 0.55, 'colsample_bytree': 0.8500000000000001, 'early_stoppig_rounds': 92}. Best is trial 36 with value: 153069.70175736025.


Avg RMSE: 180531.56852194102


In [14]:
study_xgb.best_value

153069.70175736025

In [15]:
study_xgb.best_params

{'n_estimators': 268,
 'max_depth': 6,
 'learning_rate': 0.08085988097537695,
 'min_child_weight': 2,
 'gamma': 0.0002391802642514539,
 'subsample': 0.55,
 'colsample_bytree': 1.0,
 'early_stoppig_rounds': 97}

In [17]:
tuned_xgb_params = {'n_estimators': 268,
                         'max_depth': 6,
                         'learning_rate': 0.08085988097537695,
                         'min_child_weight': 2,
                         'gamma': 0.0002391802642514539,
                         'subsample': 0.55,
                         'colsample_bytree': 1.0,
                         'early_stoppig_rounds': 97}

In [19]:
# we need a validation set, but only of those samples that are from teh competition dataset
X_t, X_val, y_t, y_val = train_test_split(X, y, 
                                          shuffle=True, random_state=1337,
                                         test_size=0.15)

X_combined.drop(X_val.index, inplace=True)
y_combined.drop(y_val.index, inplace=True)

In [20]:
xgb_model = xgb.XGBRegressor(**tuned_xgb_params)
xgb_model.fit(X_t, y_t,
              eval_set=[(X_val, y_val)],
              eval_metric="rmse", verbose=False)

[19:33:32] WARNING: ../src/learner.cc:627: 
Parameters: { "early_stoppig_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1.0,
             early_stoppig_rounds=97, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None,
             gamma=0.0002391802642514539, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.08085988097537695, max_bin=256,
             max_cat_to_onehot=4, max_delta_step=0, max_depth=6, max_leaves=0,
             min_child_weight=2, missing=nan, monotone_constraints='()',
             n_estimators=268, n_jobs=0, num_parallel_tree=1, predictor='auto',
             random_state=0, reg_alpha=0, ...)

In [21]:
y_preds = xgb_model.predict(X_test)

In [22]:
submission = pd.DataFrame({'id': test_idx, 'price': y_preds})
submission.head()

,id,price
0,22730,4769977.00
1,22731,6227576.00
2,22732,9072999.00
3,22733,1601274.75
4,22734,6754006.50


In [23]:
submission.to_csv("submission.csv", index=False)